## Supervised Machine Learning - Support Vector Machines (Star vs. QSO)

### Dependencies:

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [2]:
columns = ["obj_ID",'alpha','delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'redshift',
       'plate', 'MJD', 'fiber_ID']
target = ["class"]

In [3]:
# # Create engine
# engine = create_engine("sqlite:///

In [4]:
# # reflect an existing database into a new model
# Base = automap_base()
# # reflect the tables
# Base.prepare(engine, reflect=True)
# # Save references to each table
# Measurement = Base.classes.measurement
# Station = Base.classes.station

In [5]:
# # Create our session (link) from Python to the DB
# session = Session(engine)

In [6]:
# Load the data
file_path = Path('Resources/star_qso.csv', index=False)
data_df = pd.read_csv(file_path)
# df = df.loc[:, columns].copy()
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237680e+18,340.995120,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,8102,301,3,110,5.658980e+18,QSO,1.424659,5026,55855,741
1,1.237680e+18,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,7773,301,2,462,1.250000e+19,QSO,0.586455,11069,58456,113
2,1.237670e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,5934,301,4,122,2.751760e+18,STAR,-0.000008,2444,54082,232
3,1.237680e+18,345.801874,32.672868,23.17274,20.14496,19.41948,19.22034,18.89359,8157,301,2,38,7.323010e+18,STAR,0.000072,6504,56540,574
4,1.237680e+18,353.201522,3.080796,24.54890,21.44267,20.95315,20.79360,20.48442,7712,301,5,284,4.822280e+18,STAR,-0.000429,4283,55864,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26221,1.237680e+18,353.951560,19.899651,21.61376,21.72610,21.39746,21.15175,21.44469,8096,301,4,190,8.559290e+18,QSO,1.954003,7602,56954,716
26222,1.237660e+18,128.395984,25.467238,19.77835,19.37797,19.28145,19.12017,19.05171,4335,301,5,123,2.170890e+18,QSO,0.465272,1928,53327,562
26223,1.237660e+18,247.224915,38.037879,21.81185,21.44972,21.19963,21.36909,20.80581,3225,301,2,246,1.207990e+19,QSO,0.943422,10729,58248,324
26224,1.237660e+18,136.329001,6.426314,18.96920,18.76108,18.46569,18.48491,18.35552,3031,301,3,109,1.342220e+18,QSO,1.029967,1192,52649,530


In [7]:
data_df.set_index(['spec_obj_ID'], inplace = True)

In [8]:
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,class,redshift,plate,MJD,fiber_ID
spec_obj_ID,,,,,,,,,,,,,,,,,
5.658980e+18,1.237680e+18,340.995120,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,8102,301,3,110,QSO,1.424659,5026,55855,741
1.250000e+19,1.237680e+18,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,7773,301,2,462,QSO,0.586455,11069,58456,113
2.751760e+18,1.237670e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,5934,301,4,122,STAR,-0.000008,2444,54082,232
7.323010e+18,1.237680e+18,345.801874,32.672868,23.17274,20.14496,19.41948,19.22034,18.89359,8157,301,2,38,STAR,0.000072,6504,56540,574
4.822280e+18,1.237680e+18,353.201522,3.080796,24.54890,21.44267,20.95315,20.79360,20.48442,7712,301,5,284,STAR,-0.000429,4283,55864,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8.559290e+18,1.237680e+18,353.951560,19.899651,21.61376,21.72610,21.39746,21.15175,21.44469,8096,301,4,190,QSO,1.954003,7602,56954,716
2.170890e+18,1.237660e+18,128.395984,25.467238,19.77835,19.37797,19.28145,19.12017,19.05171,4335,301,5,123,QSO,0.465272,1928,53327,562
1.207990e+19,1.237660e+18,247.224915,38.037879,21.81185,21.44972,21.19963,21.36909,20.80581,3225,301,2,246,QSO,0.943422,10729,58248,324


In [9]:
 new_data_df=data_df.drop(['alpha','delta','run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'MJD', 'fiber_ID'], axis = 1 )

In [10]:
new_data_df

,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,
5.658980e+18,23.48827,23.33776,21.32195,20.25615,19.54544,QSO,1.424659
1.250000e+19,21.46973,21.17624,20.92829,20.60826,20.42573,QSO,0.586455
2.751760e+18,21.74669,20.03493,19.17553,18.81823,18.65422,STAR,-0.000008
7.323010e+18,23.17274,20.14496,19.41948,19.22034,18.89359,STAR,0.000072
4.822280e+18,24.54890,21.44267,20.95315,20.79360,20.48442,STAR,-0.000429
...,...,...,...,...,...,...,...
8.559290e+18,21.61376,21.72610,21.39746,21.15175,21.44469,QSO,1.954003
2.170890e+18,19.77835,19.37797,19.28145,19.12017,19.05171,QSO,0.465272
1.207990e+19,21.81185,21.44972,21.19963,21.36909,20.80581,QSO,0.943422


### Variable engineering

In [11]:
frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_u"] = pd.cut(new_data_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_g"] = pd.cut(new_data_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
new_data_df["binned_r"] = pd.cut(new_data_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
new_data_df["binned_i"] = pd.cut(new_data_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
new_data_df["binned_z"] = pd.cut(new_data_df["z"], z_size_bins, labels = frequency)

new_data_df.sample(30)

,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,
3.573660e+18,23.94946,21.21188,19.62310,18.73954,18.26392,STAR,-0.000251,3,3,2,2,3
2.131380e+18,20.02059,18.93423,18.86429,18.91162,18.95932,STAR,-0.000488,3,3,2,2,3
4.255030e+18,22.73003,22.07831,21.51227,21.18913,20.65965,QSO,2.354672,3,3,2,2,3
6.800700e+18,17.73000,17.59976,17.95371,18.19820,18.49835,STAR,0.000135,3,3,2,2,3
9.220000e+18,21.77220,21.25971,20.98307,20.44834,20.28928,QSO,0.629209,3,3,2,2,3
6.620410e+18,20.35633,19.56136,19.29128,19.20524,18.96964,QSO,2.588670,3,3,2,2,3
4.772890e+18,22.41370,21.60423,21.38309,21.10791,21.14821,QSO,0.863480,3,3,2,2,3
1.302120e+19,23.07982,25.27933,23.08398,21.71516,19.82993,QSO,1.724256,3,3,3,2,3
3.091830e+18,21.94497,19.44920,17.92686,16.55111,15.82048,STAR,-0.000029,3,3,2,1,3


### Test and Train Data

In [12]:
# Create our features
X = new_data_df.drop('class', axis=1)

# Create our target
y = new_data_df['class']

print(X.shape)
print(y.shape)

(26226, 11)
(26226,)


In [13]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
Counter(y_train)

Counter({'QSO': 9103, 'STAR': 10566})

### Random Oversampling

In [14]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'QSO': 10566, 'STAR': 10566})

### Support Vector Machine (Star vs. QSO) - Oversampling

In [15]:
# Train the Support Vector Machine Model using the resampled data
model = SVC(kernel='linear')
model.fit(X_resampled, y_resampled)

SVC(kernel='linear')

In [16]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9985168094924193

In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        QSO       1.00      1.00      1.00      1.00      1.00      1.00      3034
       STAR       1.00      1.00      1.00      1.00      1.00      1.00      3523

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      6557



In [18]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cmros = pd.DataFrame(c, index=["Actual - Galaxy", "Actual - QSO"], columns=["Predicted - Galaxy", "Predicted - QSO"])
cmros

,Predicted - Galaxy,Predicted - QSO
Actual - Galaxy,3025,9
Actual - QSO,0,3523


### Random Undersampling

In [19]:
# Resample the training data with the RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled1, y_resampled1 = rus.fit_resample(X_train, y_train)
Counter(y_resampled1)

Counter({'QSO': 9103, 'STAR': 9103})

### Support Vector Machine (Star vs. QSO) - Undersampling

In [20]:
model2 = SVC(kernel='linear')
model2.fit(X_resampled1, y_resampled1)

SVC(kernel='linear')

In [21]:
# Calculated the balanced accuracy score
y_pred1 = model2.predict(X_test)
balanced_accuracy_score(y_test, y_pred1)

0.9985168094924193

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred1))

                   pre       rec       spe        f1       geo       iba       sup

        QSO       1.00      1.00      1.00      1.00      1.00      1.00      3034
       STAR       1.00      1.00      1.00      1.00      1.00      1.00      3523

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      6557



In [23]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred1)
cmrus = pd.DataFrame(c, index=["Actual - Galaxy", "Actual - QSO"], columns=["Predicted - Galaxy", "Predicted - QSO"])
cmrus

,Predicted - Galaxy,Predicted - QSO
Actual - Galaxy,3025,9
Actual - QSO,0,3523
